## Notebook setup

In [ ]:
import io
import warnings
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

from gamestonk_terminal import api as gst
from gamestonk_terminal.helper_classes import TerminalStyle

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
warnings.filterwarnings("ignore")

In [ ]:
try:
    theme = TerminalStyle("light", "light", "light")
except:
    pass
stylesheet = gst.widgets.html_report_stylesheet()

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
ticker = "AMC"
report_name = ""

In [ ]:
ticker_data = gst.stocks.load(ticker, start=datetime.now() - timedelta(days=365))
ticker_data = gst.stocks.process_candle(ticker_data)

report_title = (f"{ticker.upper()} Due Diligence report ({datetime.now().strftime('%Y-%m-%d %H:%M:%S')})")
report_title

In [ ]:
overview = gst.stocks.fa.models.yahoo_finance.get_info(ticker=ticker).transpose()[
    "Long business summary"
][0]
overview

## Data

In [ ]:
(
    df_year_estimates,
    df_quarter_earnings,
    df_quarter_revenues,
) = gst.stocks.dd.models.business_insider.get_estimates(ticker)

### 1. Yearly Estimates

In [ ]:
display_year = sorted(df_year_estimates.columns.tolist())[:3]
df_year_estimates = df_year_estimates[display_year].head(5)
df_year_estimates

### 2. Quarterly Earnings

In [ ]:
df_quarter_earnings

### 3. Quarterly Revenues

In [ ]:
df_quarter_revenues

### 4. SEC Filings

In [ ]:
df_sec_filings = gst.stocks.dd.models.marketwatch.get_sec_filings(ticker)[
    ["Type", "Category", "Link"]
].head(5)
df_sec_filings

### 5. Analyst Ratings

In [ ]:
df_analyst = gst.stocks.dd.models.finviz.get_analyst_data(ticker)
df_analyst["target_to"] = df_analyst["target_to"].combine_first(df_analyst["target"])
df_analyst = df_analyst[["category", "analyst", "rating", "target_to"]].rename(
    columns={
        "category": "Category",
        "analyst": "Analyst",
        "rating": "Rating",
        "target_to": "Price Target",
    }
)
df_analyst

## Plots

### 1. Price history

In [ ]:
fig, (candles, volume) = plt.subplots(nrows=2, ncols=1, figsize=(5, 3), dpi=150)
gst.stocks.candle(
    s_ticker=ticker,
    df_stock=ticker_data,
    use_matplotlib=True,
    external_axes=[candles, volume],
)
candles.set_xticklabels("")
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
price_chart = f.getvalue().decode("utf-8")

### 2. Price Target

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3), dpi=150)
gst.stocks.dd.pt(
    ticker=ticker,
    start="2021-10-25",
    interval="1440min",
    stock=ticker_data,
    num=10,
    raw=False,
    external_axes=[ax],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
price_target_chart = f.getvalue().decode("utf-8")

### 3. Ratings over time

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3), dpi=150)
gst.stocks.dd.rot(
    ticker=ticker,
    num=10,
    raw=False,
    export="",
    external_axes=[ax],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
ratings_over_time_chart = f.getvalue().decode("utf-8")

## Render the report template to a file

In [ ]:
body = ""

In [ ]:
# Title
body += gst.widgets.h(1, report_title)

body += gst.widgets.h(2, "Overview")
body += gst.widgets.row([gst.widgets.p(overview)])


# Analysts ratings
body += gst.widgets.h(2, "Analyst assessments")
body += gst.widgets.row([price_target_chart])
body += gst.widgets.row([df_analyst.to_html()])
body += gst.widgets.row([ratings_over_time_chart])

# Price history and yearly estimates
body += gst.widgets.row(
    [
        gst.widgets.h(3, "Price history") + price_chart,
        gst.widgets.h(3, "Estimates") + df_year_estimates.head().to_html(),
    ]
)

# Earnings and revenues
body += gst.widgets.h(2, "Earnings and revenues")
body += gst.widgets.row([df_quarter_earnings.head().to_html()])
body += gst.widgets.row([df_quarter_revenues.head().to_html()])

# Sec filings and insider trading
body += gst.widgets.h(2, "SEC filings")
body += gst.widgets.row([df_sec_filings.to_html()])

In [ ]:
report = gst.widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

In [ ]:
# to save the results
with open(report_name + ".html", "w") as fh:
    fh.write(report)